# Claude CLI - Automatisation Avancee

**Module :** Vibe-Coding / Claude Code / Notebooks CLI  
**Niveau :** Avance  
**Duree :** 30 min  
**Prerequis :** Notebooks 01-04 completes

## Objectifs d'Apprentissage

- [ ] Creer des pipelines d'automatisation avec subprocess
- [ ] Integrer Claude dans des scripts Bash/PowerShell
- [ ] Comprendre les slash commands (/commit, /review)
- [ ] Decouvrir les hooks (PreToolUse, PostToolUse)
- [ ] Construire un script de revue de code automatise

---

## 1. Configuration

In [ ]:
import sys
import subprocess
import json
import os
from pathlib import Path

sys.path.insert(0, 'helpers')
from claude_cli import run_claude, verify_installation, print_response

EXAMPLES_DIR = Path('examples/sample_project')
print(f"Claude CLI pret: {verify_installation()}")

## 2. Pipelines avec Subprocess

Les pipelines permettent de chainer des operations Claude pour des workflows complexes :

```
Input → [Claude: Analyse] → [Claude: Correction] → [Claude: Review] → Output
```

In [ ]:
def create_pipeline(*steps):
    """
    Cree un pipeline de prompts Claude.
    
    Args:
        steps: Liste de tuples (nom, prompt_template)
        
    Returns:
        Fonction qui execute le pipeline.
    """
    def execute(initial_input):
        current_data = initial_input
        results = []
        
        for step_name, prompt_template in steps:
            prompt = prompt_template.format(input=current_data)
            stdout, stderr, code = run_claude(prompt, model="haiku")
            
            if code != 0:
                return {"error": f"Etape '{step_name}' echouee", "stderr": stderr}
            
            results.append({"step": step_name, "output": stdout})
            current_data = stdout
        
        return {"success": True, "results": results, "final_output": current_data}
    
    return execute

print("Fonction create_pipeline definie")

In [ ]:
# Exemple de pipeline : Analyse → Resume → Ameliorations
code_review_pipeline = create_pipeline(
    ("analyse", "Analyse ce code et liste ses caracteristiques principales:\n{input}"),
    ("resume", "Resume cette analyse en 3 points cles:\n{input}"),
    ("ameliorations", "Propose 2 ameliorations basees sur ce resume:\n{input}")
)

print("Pipeline de revue de code defini")

In [ ]:
# Test du pipeline (ATTENTION: 3 appels API)
sample_code = '''
def process_data(items):
    result = []
    for item in items:
        if item > 0:
            result.append(item * 2)
    return result
'''

# Decommentez pour executer
# output = code_review_pipeline(sample_code)
# if output.get("success"):
#     for r in output["results"]:
#         print(f"\n=== {r['step'].upper()} ===")
#         print(r['output'][:300])

print("(Pipeline non execute - decommentez pour tester)")

## 3. Integration Scripts Shell

Claude Code peut etre integre dans des scripts Bash ou PowerShell.

### Exemple Bash
```bash
#!/bin/bash
# Script de revue automatique

FILE=$1
REVIEW=$(claude -p "Revue de code pour: $(cat $FILE)")
echo "$REVIEW" > "${FILE}.review.md"
```

### Exemple PowerShell
```powershell
# Analyse de tous les fichiers Python
Get-ChildItem -Filter *.py | ForEach-Object {
    $content = Get-Content $_.FullName -Raw
    claude -p "Resume ce fichier: $content"
}
```

In [ ]:
# Generer un script de revue automatique
bash_script = '''#!/bin/bash
# Script de revue de code automatique avec Claude

FILE="$1"

if [ -z "$FILE" ]; then
    echo "Usage: $0 <fichier.py>"
    exit 1
fi

if [ ! -f "$FILE" ]; then
    echo "Fichier non trouve: $FILE"
    exit 1
fi

echo "Analyse de $FILE..."
CONTENT=$(cat "$FILE")

REVIEW=$(claude -p "Effectue une revue de code pour ce fichier Python. 
Identifie: bugs potentiels, problemes de style, ameliorations possibles.

Code:
$CONTENT")

OUTPUT="${FILE%.py}.review.md"
echo "# Code Review: $FILE" > "$OUTPUT"
echo "" >> "$OUTPUT"
echo "$REVIEW" >> "$OUTPUT"

echo "Revue sauvegardee dans: $OUTPUT"
'''

print(bash_script)

In [ ]:
# Version Python equivalente
def auto_review(filepath):
    """Effectue une revue de code automatique."""
    path = Path(filepath)
    
    if not path.exists():
        return {"error": f"Fichier non trouve: {filepath}"}
    
    content = path.read_text()
    
    stdout, stderr, code = run_claude(
        f"""Effectue une revue de code pour ce fichier Python.
Identifie:
1. Bugs potentiels
2. Problemes de style (PEP 8)
3. Ameliorations possibles
4. Securite

Format: sections avec titres markdown.

Code:
{content}""",
        model="sonnet"
    )
    
    if code == 0:
        output_path = path.with_suffix('.review.md')
        output_path.write_text(f"# Code Review: {path.name}\n\n{stdout}")
        return {"success": True, "review": stdout, "output_file": str(output_path)}
    else:
        return {"error": stderr}

print("Fonction auto_review definie")

## 4. Slash Commands

Claude Code propose des commandes integrees accessibles avec `/` :

| Commande | Description |
|----------|-------------|
| `/init` | Genere un CLAUDE.md pour le projet |
| `/commit` | Cree un commit avec message genere |
| `/review` | Revue des changements en cours |
| `/status` | Statut de connexion |
| `/mcp` | Statut des serveurs MCP |

Ces commandes sont utilisees en mode interactif (`claude` sans `-p`).

In [ ]:
# Simuler /commit : generer un message de commit
def generate_commit_message(diff_content):
    """Genere un message de commit a partir d'un diff."""
    stdout, stderr, code = run_claude(
        f"""Genere un message de commit conventionnel pour ces changements.

Format attendu:
type(scope): description courte

Description detaillee si necessaire.

Types: feat, fix, docs, style, refactor, test, chore

Diff:
{diff_content}""",
        model="haiku"
    )
    return stdout if code == 0 else f"Erreur: {stderr}"

# Exemple de diff
example_diff = '''
--- a/utils.py
+++ b/utils.py
@@ -15,6 +15,10 @@ def calculate_statistics(data):
+    if not data:
+        raise ValueError("Empty data")
+
     mean = sum(data) / len(data)
'''

# Decommentez pour tester
# message = generate_commit_message(example_diff)
# print(message)

print("Fonction generate_commit_message definie")

## 5. Hooks (Concept)

Les **hooks** permettent d'intercepter les actions de Claude pour les modifier ou les valider.

Types de hooks :
- **PreToolUse** : Avant l'utilisation d'un outil (Write, Bash, etc.)
- **PostToolUse** : Apres l'utilisation
- **Notification** : Sur certains evenements

Configuration dans `.claude/settings.json` :

```json
{
  "hooks": {
    "PreToolUse": {
      "Write": "python validate_write.py"
    },
    "PostToolUse": {
      "Bash": "python log_bash.py"
    }
  }
}
```

In [ ]:
# Exemple de hook de validation
def validate_write_hook(filepath, content):
    """Hook qui valide le contenu avant ecriture."""
    issues = []
    
    # Verifications de securite
    if 'password' in content.lower() and '=' in content:
        issues.append("Possible mot de passe en clair detecte")
    
    if 'api_key' in content.lower():
        issues.append("Possible cle API detectee")
    
    # Verifications de style
    if filepath.endswith('.py'):
        if 'import *' in content:
            issues.append("Import wildcard detecte (mauvaise pratique)")
    
    return {
        "approved": len(issues) == 0,
        "issues": issues
    }

# Test
test_content = '''
from utils import *
api_key = "sk-1234"
'''

result = validate_write_hook("config.py", test_content)
print(f"Approuve: {result['approved']}")
print(f"Problemes: {result['issues']}")

## 6. Script de Revue Automatise Complet

In [ ]:
class CodeReviewer:
    """Revue de code automatisee avec Claude."""
    
    def __init__(self, model="sonnet"):
        self.model = model
        self.checks = [
            ("bugs", "Identifie les bugs potentiels"),
            ("style", "Verifie le style PEP 8"),
            ("security", "Analyse les problemes de securite"),
            ("performance", "Identifie les optimisations possibles"),
        ]
    
    def review_file(self, filepath):
        """Effectue une revue complete d'un fichier."""
        path = Path(filepath)
        if not path.exists():
            return {"error": "Fichier non trouve"}
        
        content = path.read_text()
        results = {"file": str(path), "checks": {}}
        
        for check_name, check_desc in self.checks:
            stdout, stderr, code = run_claude(
                f"{check_desc} dans ce code (reponse courte):\n\n{content}",
                model="haiku"  # Rapide pour chaque check
            )
            results["checks"][check_name] = stdout if code == 0 else f"Erreur: {stderr}"
        
        return results
    
    def review_project(self, directory, pattern="*.py"):
        """Revue tous les fichiers d'un projet."""
        path = Path(directory)
        files = list(path.rglob(pattern))
        
        return {
            "directory": str(path),
            "files_count": len(files),
            "files": [str(f) for f in files]
            # En production, on appellerait review_file pour chaque fichier
        }
    
    def generate_report(self, results):
        """Genere un rapport markdown."""
        lines = [f"# Code Review: {results['file']}\n"]
        
        for check, output in results.get("checks", {}).items():
            lines.append(f"## {check.title()}\n")
            lines.append(output + "\n")
        
        return "\n".join(lines)

print("CodeReviewer defini")

In [ ]:
# Test du reviewer (ATTENTION: multiple appels API)
reviewer = CodeReviewer()

# Voir les fichiers disponibles
project_info = reviewer.review_project(EXAMPLES_DIR)
print(f"Fichiers dans le projet: {project_info['files_count']}")
for f in project_info['files']:
    print(f"  - {f}")

In [ ]:
# Revue d'un fichier (decommentez pour executer - 4 appels API)
# results = reviewer.review_file(EXAMPLES_DIR / "utils.py")
# report = reviewer.generate_report(results)
# print(report)

print("(Revue non executee - decommentez pour tester)")

## 7. Exercices

In [ ]:
# EXERCICE 1 : Creez un pipeline qui:
# 1. Lit un fichier Python
# 2. Identifie les fonctions sans docstring
# 3. Genere les docstrings manquantes

# Votre code ici


In [ ]:
# EXERCICE 2 : Creez un hook qui verifie
# que tout fichier Python a au moins un test associe

# Votre code ici


In [ ]:
# EXERCICE 3 : Etendez CodeReviewer avec un check supplementaire
# de votre choix (ex: documentation, tests, complexite)

# Votre code ici


## 8. Resume Final

Dans cette serie de notebooks, nous avons appris :

| Notebook | Competences acquises |
|----------|---------------------|
| **01-Bases** | `claude -p`, modeles, formats de sortie |
| **02-Sessions** | `-c`, sessions, conversations multi-tours |
| **03-References** | @fichier, plages de lignes, CLAUDE.md |
| **04-Agents** | Explore, Plan, subagents, parallelisation |
| **05-Automatisation** | Pipelines, scripts, hooks, revue automatisee |

### Pour aller plus loin

- **Documentation officielle** : [docs.claude.com/code](https://docs.claude.com/code)
- **Concepts avances** : [CONCEPTS-AVANCES.md](../../docs/claude-code/CONCEPTS-AVANCES.md)
- **Modeles alternatifs** : [OPENROUTER-MODELES-ALTERNATIFS.md](../../docs/claude-code/OPENROUTER-MODELES-ALTERNATIFS.md)

### Ressources communautaires

- [Awesome Claude Code](https://github.com/hesreallyhim/awesome-claude-code)
- [SkillsMP Marketplace](https://skillsmp.com/)

---

**Felicitations !** Vous avez complete la serie de notebooks Claude CLI.